In [73]:
from pinecone import Pinecone
pc = Pinecone(api_key="949926ec-01d8-40c1")
index = pc.Index("test2")


In [74]:
pc.list_indexes()

{'indexes': [{'dimension': 384,
              'host': 'test2-ljwx5f4.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'test2',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [75]:
DATA_DIR_PATH="/home/sets/PycharmProjects/llm/mistral_chatbot/data"

In [77]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
dir_loader = DirectoryLoader(
    DATA_DIR_PATH,
    glob='*.txt',
    loader_cls=TextLoader
)
docs = dir_loader.load()
print("PDFs Loaded")

PDFs Loaded


In [79]:
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
txt_splitter = RecursiveCharacterTextSplitter(separators=['[splixt]'])

In [82]:
from langchain_community.embeddings import HuggingFaceEmbeddings,OpenAIEmbeddings
inp_txt = txt_splitter.split_documents(docs)

hfembeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cuda'}
)

/home/sets/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [83]:
hfembeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={}, multi_process=False, show_progress=False)

In [84]:
inp_txt = txt_splitter.split_documents(docs)

In [87]:
import os
os.environ['PINECONE_API_KEY'] = "949926ec-01d8-40c1-a59f-5da066cd75e6"
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_texts([t.page_content for t in inp_txt],hfembeddings,index_name='test2')
print(docsearch)

In [92]:
def create_llms_model():
    llm = CTransformers(model="mistral-7b-instruct-v0.1.Q4_K_M.gguf", config={'max_new_tokens': 128, 'temperature': 0.5,'context_length': 7000})
    return llm
llm = create_llms_model()


In [100]:
custom_prompt=''With the information provided try to answer the question. If you cant answer the question based 
on the information either say you cant find an answer or unable to find an answer. So try to understand the context 
and answer only based on the information provided.pPathaanvrovide true information to and deny if the question is false

Context: {context}
Question: {question}
Do provide only helpful answers

Helpful answer:
'''

SyntaxError: unterminated triple-quoted string literal (detected at line 10) (3039903043.py, line 10)

In [15]:
from sentence_transformers import SentenceTransformer

2024-05-28 17:53:23.410710: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-28 17:53:23.636189: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 17:53:24.112217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [16]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/home/sets/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [62]:
def find_match(input):
    input_em = model.encode(input).tolist()
    
    result = index.query(vector=input_em, top_k=2, includeMetadata=True)
    return result['matches'][0]['metadata']['text']+"\n"+result['matches'][1]['metadata']['text']

In [63]:
context=find_match("what is the news about loksabha candidate dharmendra yadav")

In [26]:
pip install -U langchain-community

Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.1 MB/s eta 0:00:0031m18.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 87.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [27]:
from langchain_community.llms.ctransformers import CTransformers

In [30]:
def create_llms_model():
    llm = CTransformers(model="mistral-7b-instruct-v0.1.Q4_K_M.gguf", config={'max_new_tokens': 128, 'temperature': 0.5})
    return llm
llm = create_llms_model()

In [31]:
llm

CTransformers(client=<ctransformers.llm.LLM object at 0x740a34e33c10>, model='mistral-7b-instruct-v0.1.Q4_K_M.gguf', config={'max_new_tokens': 128, 'temperature': 0.5})

In [32]:
from langchain.chains import RetrievalQA

In [131]:
retriveqa=RetrievalQA.from_chain_type(llm=llm,chain_type='stuff',retriever=docsearch.as_retriever(search_kwargs={"k": 1}))

In [ ]:
retriveqa.run(input())

In [152]:
from langchain import PromptTemplate
from langchain.memory import ConversationBufferMemory
template = """
Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) Answer the question :
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

In [153]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=docsearch.as_retriever(search_kwargs={"k": 1}),
    verbose=True,
    chain_type_kwargs={
        "verbose": False,
        "prompt": prompt,
        "memory": ConversationBufferMemory(
            memory_key="history",
            input_key="question"),
    }
)

In [155]:
qa.run({"query": "which website referred for it for fact-checking"})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) Answer the question :
------
<ctx>
[splixt]On January 24. 2023, the New York-based  Hindenburg Research brought out a report  titled Adani Group: How The World’s 3rd Richest Man Is Pulling The Largest Con In Corporate History accusing the Gautam Adani-led company of ‘stock manipulation and accounting fraud’ spanning over decades. Hindenburg Research is an investor research firm founded by  Nathan Anderson . Subsequently, the issue raised a huge row in the Parliament and the Congress launched  a nationwide protest . In this context, several Facebook users claimed that Congress leader Rahul Gandhi had met Hindenburg founder Nathan Anderson and shared a photo. Some of the posts can be seen in the following slide-show. This slideshow requires JavaScr

'Check News Fact'

In [65]:
from langchain.chains import ConversationChain
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder
)

In [119]:
system_msg_template = SystemMessagePromptTemplate.from_template(template="""Answer the question as truthfully as possible using the provided {context}, 
and if the answer is not contained within the text below, say 'I don't know'""")


human_msg_template = HumanMessagePromptTemplate.from_template(template="{input}")

In [120]:
prompt_template = ChatPromptTemplate.from_messages([system_msg_template, MessagesPlaceholder(variable_name="history"), human_msg_template])

In [121]:
prompt_template

ChatPromptTemplate(input_variables=['context', 'history', 'input'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Answer the question as truthfully as possible using the provided {context}, \nand if the answer is not contained within the text below, say 'I don't know'")), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [127]:
from langchain.chains import ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llm=llm, chain_type='stuff',
                                              retriever=docsearch.as_retriever(search_kwargs={"k": 1})
                                            )

In [107]:
pip install streamlit

Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 31.7 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 60.9 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.6 MB/s eta 0:00:00m eta 0:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [108]:
import streamlit as st
if 'history' not in st.session_state:
    st.session_state['history'] = []

if 'generated' not in st.session_state:
    st.session_state['generated'] = ["Hello! Ask me anything about 🤗"]

if 'past' not in st.session_state:
    st.session_state['past'] = ["Hey! 👋"]

2024-05-29 10:51:36.041 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


In [128]:
result = chain({"question": query, "chat_history": st.session_state['history']})

In [129]:
result

{'question': 'is the viral video of rashmika madhana in lift is true',
 'chat_history': [],
 'answer': ' No, the viral video of Rashmika Mandanna getting into an elevator is not true. It has been doctored with the help of deepfake technology to make it look like Rashmika Mandanna. The original video was of Zara Patel, a British-Indian girl with 415K followers on Instagram, and was edited by someone else.'}

In [113]:
query="is the viral video of rashmika madhana in lift is true"